develop the add existing base years per grade

In [45]:
%load_ext autoreload
%autoreload 2
%precision %e

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'%e'

In [46]:
# running the jupyter notebook on the compute nodes doesnt build the path as expected, you have to manually do this
import IPython, os

working_directory = os.path.dirname(IPython.extract_module_locals()[1]['__vsc_ipynb_file__'])
workflow_dir = os.path.dirname(working_directory)
scripts_dir = os.path.join(workflow_dir, "scripts")
root_dir = os.path.dirname(workflow_dir)

os.chdir(scripts_dir)


In [47]:
import logging
import pypsa
import os.path
from _helpers import mock_snakemake
from _pypsa_helpers import shift_profile_to_planning_year

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd

In [48]:
if "snakemake" not in globals():
    snakemake = mock_snakemake(
        "add_existing_baseyear",
        topology="current+FCG",
        snakefile_path=os.path.abspath("../"),
        co2_pathway="exp175default",
        planning_horizons="2020",
        heating_demand="positive",
    )

In [49]:
import numpy as np
from add_electricity import load_costs

In [50]:


def read_existing_capacities(paths_dict: dict) -> pd.DataFrame:
    # TODO fix/centralise ()
    carrier = {
        "coal": "coal power plant",
        "CHP coal": "CHP coal",
        "CHP gas": "CHP gas",
        "OCGT": "OCGT gas",
        "solar": "solar",
        "solar thermal": "solar thermal",
        "onwind": "onwind",
        "offwind": "offwind",
        "coal boiler": "coal boiler",
        "ground heat pump": "heat pump",
        "nuclear": "nuclear",
    }
    df_agg = pd.DataFrame()
    # TODO fix centralise (make a dict from start?)
    for tech in carrier:

        # TODO make argument
        df = pd.read_csv(paths_dict[tech], index_col=0).fillna(0.0)
        df.columns = df.columns.astype(int)
        df = df.sort_index()

        for year in df.columns:
            for node in df.index:
                name = f"{node}-{tech}-{year}"
                capacity = df.loc[node, year]
                if capacity > 0.0:
                    df_agg.at[name, "Fueltype"] = carrier[tech]
                    df_agg.at[name, "Tech"] = tech
                    df_agg.at[name, "Capacity"] = capacity
                    df_agg.at[name, "DateIn"] = year
                    df_agg.at[name, "cluster_bus"] = node

    return df_agg

In [51]:
n_years = 1
config = snakemake.config
tech_costs = snakemake.input.tech_costs
cost_year = snakemake.wildcards["planning_horizons"]
data_paths = {k: v for k, v in snakemake.input.items()}
costs = load_costs(tech_costs, config["costs"], config["electricity"], cost_year, n_years)

grouping_years = config["existing_capacities"]["grouping_years"]
existing_capacities = read_existing_capacities(data_paths)
# include renewables in df_agg
df_agg = existing_capacities.copy()

df_agg["grouping_year"] = np.take(
    grouping_years, np.digitize(df_agg.DateIn, grouping_years, right=True)
)

df = df_agg.pivot_table(
    index=["grouping_year", "Tech"], columns="cluster_bus", values="Capacity", aggfunc="sum"
)

df.fillna(0)


cluster_bus                        Anhui      Beijing  Chongqing    Fujian  \
grouping_year Tech                                                           
1985          coal                320.00     0.000000       0.00      0.00   
1990          coal                950.00     0.000000       0.00    700.00   
1995          CHP coal              0.00     0.000000       0.00      0.00   
              OCGT                  0.00     0.000000       0.00      0.00   
              coal                960.00     0.000000     720.00      0.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
2000          CHP coal            170.00     0.000000       0.00      0.00   
              CHP gas               0.00   845.000000       0.00      0.00   
              OCGT                  0.00     0.000000       0.00      0.00   
              coal               1890.00     0.000000     720.00   2500.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
              onwind                0.00     0.000000       0.00      0.00   
2005          CHP coal              0.00     0.000000       0.00    300.00   
              CHP gas               0.00     0.000000       0.00      0.00   
              OCGT                  0.00     0.000000       0.00      0.00   
              coal               2260.00     0.000000     300.00   3186.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
              ground heat pump     50.30    66.760000      47.01      5.17   
              onwind                0.00     0.000000       0.00     16.00   
2010          CHP coal            245.00     0.000000     370.00    500.00   
              CHP gas               0.00  1970.000000       0.00      0.00   
              OCGT                  0.00     0.000000       0.00   3900.00   
              coal              17640.00     0.000000    2700.00   9540.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
              ground heat pump     70.70    93.820000      66.07      7.27   
              offwind               0.00     0.000000       0.00      0.00   
              onwind                0.00   136.000000      49.00    299.00   
2015          CHP coal           1350.00     0.000000     610.00   1790.00   
              CHP gas               0.00  5820.000000     940.00      0.00   
              OCGT                  0.00     0.000000       0.00      0.00   
              coal              15530.00     0.000000    4680.00   4100.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
              ground heat pump     80.21   106.450000      74.97      8.25   
              offwind               0.00     0.000000       0.00      0.00   
              onwind             1488.00    50.000000     246.00   1176.00   
              solar              1208.00    76.000000       0.00    129.00   
2020          CHP coal            230.00     0.000000     504.00     60.00   
              CHP gas             120.00  1492.000000       0.00      0.00   
              OCGT                  0.00     0.000000       0.00      0.00   
              coal               8000.00     0.000000    2520.00   4640.00   
              coal boiler          42.00  6705.166667       0.00      0.00   
              ground heat pump     70.70    93.820000      66.07      7.27   
              nuclear               0.00     0.000000       0.00  11034.00   
              offwind               0.00     0.000000       0.00    627.00   
              onwind             2043.00     0.000000     528.00   2006.00   
              solar             12492.00   544.000000     670.00   1891.00   
              solar thermal         0.00     0.000000       0.00      0.00   

cluster_bus                           Gansu  Guangdong  Guangxi    Guizhou  \
grouping_year Tech                                                           
1985          coal   

In [63]:
df.loc[(slice(None),"onwind"),:]

,cluster_bus,Anhui,Beijing,Chongqing,Fujian,Gansu,Guangdong,Guangxi,Guizhou,Hainan,Hebei,...,Shaanxi,Shandong,Shanghai,Shanxi,Sichuan,Tianjin,Tibet,Xinjiang,Yunnan,Zhejiang
grouping_year,Tech,,,,,,,,,,,,,,,,,,,,,
2000,onwind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,onwind,NaN,NaN,NaN,16.0,NaN,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,onwind,NaN,136.0,49.0,299.0,1190.0,517.2,NaN,NaN,198.0,2724.0,...,NaN,1246.0,15.0,373.0,NaN,12.0,NaN,700.0,194.0,138.0
2015,onwind,1488.0,50.0,246.0,1176.0,9950.5,1562.1,421.0,2606.0,50.0,6853.0,...,1638.0,6467.0,355.0,5899.0,828.0,309.0,49.5,10905.0,4637.0,844.0
2020,onwind,2043.0,NaN,528.0,2006.0,1661.0,2528.0,4112.0,1914.0,NaN,9270.5,...,5094.0,9107.0,NaN,10776.4,3103.0,467.0,NaN,9799.0,3657.0,338.0


In [ ]:
import xarray as xr

with xr.open_dataset(prof_paths[f"profile_{tech}"]) as ds:
    if ds.indexes["bus"].empty:
        continue
    if "year" in ds.indexes:
        ds = ds.sel(year=ds.year.min(), drop=True)

TypeError: digitize() missing 2 required positional arguments: 'x' and 'bins'

In [ ]:
existing_capacities.query("Tech == 'coal'").Capacity.sum()

np.float64(788949.2)

In [ ]:


n = pypsa.Network(snakemake.input.network)
n_years = n.snapshot_weightings.generators.sum() / 8760


baseyear = snakemake.params["baseyear"]
# add_build_year_to_new_assets(n, baseyear)


add_power_capacities_installed_before_baseyear(
    n, grouping_years, costs, baseyear, config, df
)
n.export_to_netcdf(snakemake.output[0])

logger.info("Existing capacities successfully added to network")


/home/ivanra/miniconda3/envs/pypsa-china/lib/python3.12/site-packages/pypsa/components.py:323: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/home/ivanra/miniconda3/envs/pypsa-china/lib/python3.12/site-packages/pypsa/components.py:323: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/home/ivanra/miniconda3/envs/pypsa-china/lib/python3.12/site-packages/pypsa/components.py:323: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/home/ivanra/miniconda3/envs/pypsa-china/li

FileNotFoundError: [Errno 2] No such file or directory: '/home/ivanra/documents/PyPSA-China-PIK/results/v-0.2.0_unamed_run/overnight_co2pw-exp175default_topo-current+FCG_proj-positive/prenetworks/ntwk_2020.nc'